In [1]:
import pathlib
import numpy as np

import matplotlib.pyplot as plt

import torch
from torch import nn

import datasets
from transformers import AutoTokenizer

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [37]:
dataset = datasets.load_dataset('bookcorpus')
dataset['train'][:3]

{'text': ['usually , he would be tearing around the living room , playing with his toys .',
  'but just one look at a minion sent him practically catatonic .',
  "that had been megan 's plan when she got him dressed earlier ."]}

In [33]:
# import model.rotator_lima3 as rotator
# subpath = '20240422.11:52:35-batch_size_1024'

# import model.rotator_multihead_local_coordinate as rotator
# subpath = '20240422.02:23:48-batch_size_1024'
# batch_num = 10000

import model.rotator_multireslima_rotating_history as rotator
subpath = '20240422.02:12:51-batch_size_1024' #for report
subpath = '20240422.11:49:56-batch_size_1024' #rotator_lima3, dim96, head12, depth3

batch_num = 10000

In [34]:
folder_path = pathlib.Path('data/limanet/')
model = torch.load(folder_path/subpath/f'batch_{batch_num}-model.pt').cpu()

In [35]:
model.predictor.rotary_denom

Parameter containing:
tensor(0.5746, requires_grad=True)

In [38]:
samples = dataset['train'][10:15]['text']

input_ids = tokenizer(samples, padding=True, return_tensors='pt', add_special_tokens=False)['input_ids']
print(input_ids)

tensor([[ 2169,  2051,  2016,  2246,  2046,  6701,  1005,  1055,  2227,  1010,
          2016,  2001,  8794,  2008,  2002,  2246,  2498,  2066,  2010,  2269,
          1012,     0,     0,     0,     0],
        [ 2010,  8899,  8855,  2606,  1998,  2630,  2159,  2020,  3294,  5106,
          1012,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [ 2009,  2001,  2069,  2010,  3857,  2008,  2002,  2001,  2635,  2044,
          2010,  2269,  1012,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [ 2073, 12756,  2001,  1037, 11737,  2378, 28546,  1019,  1005,  1017,
          1005,  1005,  1010,  4482,  2001,  1020,  1005,  1015,  1005,  1005,
          1998,  2048,  3634,  7038,  1012],
        [ 6701,  2001,  2525, 25719,  2125,  1996,  6093,  1999,  4578,  1998,
          3635,  2429,  2000,  2010, 23614,  2937,  1012,     0,     0,     0,
             0,     0,     0, 

In [39]:
# # for early non-multihead limanet

# using_distance = 'L2'

# histories = model.initial_history
# for l in range(input_ids.shape[1]):
#     predicts, histories = model.inference(input_ids[:, l], histories, return_updated_history=True, distance=using_distance)
#     tokenizer.batch_decode(predicts[:, :5])
    
#     for predict, input_id in zip(predicts, input_ids):
#         candidates = tokenizer.convert_ids_to_tokens(predict[:5].detach().numpy())
#         feeded = tokenizer.decode(input_id[:l+1])
#         print(f"{feeded} : {candidates=}")

#     print()

In [40]:
using_distance = 'L2'

histories = [history.clone() for history in model.initial_history]
for l in range(input_ids.shape[1]):
    
    predicts, histories = model.inference(input_ids[:, l], histories, l, return_updated_history=True, distance=using_distance)
    tokenizer.batch_decode(predicts[:, :5])
    
    for predict, input_id in zip(predicts, input_ids):
        candidates = tokenizer.convert_ids_to_tokens(predict[:5].detach().numpy())
        feeded = tokenizer.decode(input_id[:l+1])
        print(f"{feeded} : {candidates=}")

    print()

each : candidates=['each', 'made', 'were', 'be', 'go']
his : candidates=['them', 'voice', 'made', 'with', '!']
it : candidates=['made', 'i', 'of', "'", 'was']
where : candidates=['made', 'if', 'never', 'off', 'and']
mason : candidates=['##ing', 'never', 'found', 'be', 'could']

each time : candidates=['found', 'did', 'voice', 'off', 'if']
his platinum : candidates=['if', 'everyone', 'did', 'go', 'could']
it was : candidates=['doing', 'all', 'n', 'a', 'were']
where megan : candidates=['was', 'sure', 'teeth', 'hand', 'hands']
mason was : candidates=['all', 'an', 'off', 'if', 'a']

each time she : candidates=['made', 'back', 'if', 'they', 'few']
his platinum blond : candidates=['life', 's', 'right', 'made', 'go']
it was only : candidates=['be', 'made', 'reason', 't', 'found']
where megan was : candidates=['made', 'off', 'two', 'life', 'one']
mason was already : candidates=['made', 'back', 'there', 'were', 'thing']

each time she looked : candidates=['off', 'why', 'thing', 'this', 'two']
h

In [10]:
input_ids[:, l].shape

torch.Size([5])